##### Import Package

In [82]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from konlpy.tag import Twitter

import pandas as pd

##### Get Article

In [83]:
df_list = []
for day in range(1,31):
    day = "0" + str(day) if 10 > day else str(day)
    date = "2016-06-" + day
    df = pd.read_csv("./news/" + date + ".csv").fillna("-")
    df_list.append(df)

article_df = pd.concat(df_list).reset_index(drop=True)

len(article_df), article_df.columns

(56035,
 Index(['newsid', 'oid', 'newspaper', 'title', 'link', 'comment', 'likeit',
        'content', 'date', 'category'],
       dtype='object'))

##### Split Tratin, Test

In [84]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size=0.01, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(55474, 561, 55474, 561)

##### Set Stop Words

In [85]:
stop_words_list = ["동아일보", "경향신문"]

##### Set Morpheme

In [86]:
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

##### Set Model

In [87]:
clf = Pipeline([
#             ('vect', TfidfVectorizer(tokenizer=tokenize_pos, stop_words=stop_words, ngram_range=(1,2))),
            ('vect', TfidfVectorizer()),
            ('clf', MultinomialNB(alpha=0.01)),
        ])

In [88]:
%time model = clf.fit(X_train[:10000],y_train[:10000]) 

CPU times: user 10.4 s, sys: 939 ms, total: 11.3 s
Wall time: 11.5 s


##### Confusion Matrix, Classfication Report
- category - 0:정치, 1:경제, 2:사회, 3.생활/문화, 4.세계, 5.IT/과학

In [89]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%time y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

CPU times: user 378 ms, sys: 64.5 ms, total: 442 ms
Wall time: 449 ms
[[108   2   4   0   0   0]
 [  1  96   9   1   1   1]
 [  6   8 148   3   1   2]
 [  1   5   9  37   1   2]
 [  3   2   5   0  73   0]
 [  1   2   1   1   0  27]]
             precision    recall  f1-score   support

          0       0.90      0.95      0.92       114
          1       0.83      0.88      0.86       109
          2       0.84      0.88      0.86       168
          3       0.88      0.67      0.76        55
          4       0.96      0.88      0.92        83
          5       0.84      0.84      0.84        32

avg / total       0.87      0.87      0.87       561



In [90]:
X_test = X_test.reset_index(drop=True)
X_test

0      [뉴스데스크]◀ 앵커 ▶브렉시트의 후폭풍은 경제뿐 아니라 다방면에서 이어지고 있습니...
1      전북경찰청, 돼지에 착생한 생물 생장 정도로 사망시점 추정    (전주=연합뉴스) ...
2      산업부, 가이드라인 초안 마련철강·조선 사업재편 속도낼 듯산업계 스스로 구조조정을 ...
3      "1980년식으로 할 바엔 하지 말아야…구조조정 전 책임소재 밝혀야"    (서울=...
4      [뉴스데스크]◀ 앵커 ▶중국 어선들은 우리 서해뿐만 아니라 멀리 지구 반대편까지 가...
5       ■ 방송 : MBC 라디오 표준FM 95.9 <신동호의 시선집중>(06:15~08...
6      [동아일보][대법원 판결성향지수 첫 분석]前現 대법관 35명 이념지수 보니“남성적 ...
7      여행은 각자의 몫이다. 일반적인 여행 정보에 나만의 레시피를 더하는 것, 그것이 바...
8      "화웨이 타격 입을 수 있지만 中 반격에 한국까지 역풍 우려""中 철강 美서 밀려나...
9      【 앵커멘트 】  이번에 문제가 된 학교전담경찰관들은 학생들한테 인기가 많다고 합니...
10     © News1 최진모 디자이너(서울=뉴스1) 윤진희 기자 = 최유정 변호사와 홍만표...
11     최근 북한정권의 주민들을 대상으로 개인부동산 소유를 허용했다고 북한전문매체인 뉴포커...
12     올 하반기에 주력 산업 중 건설과 석유화학을 제외하고는 부진할 것이라는 전망이 나왔...
13     회생절차 개시 맞춰 담화문 발표 【서울=뉴시스】 한상연 기자 = 이병모 STX조선해...
14     1일부터 장마전선 본격 북상장마가 시작된 지 열흘이 지났지만 남쪽에는 집중호우가, ...
15     [머니투데이 이경은 기자] 서울중앙지검/사진=뉴스1가습기 살균제 사망 사건을 수사 ...
16     -서울시 대학생 480명 선발 1만1759명 몰려-7월 1일 업무 투입…자치구마다 ...
17     “박 대통령 못지킬 공약 왜했나” 대구, 용

In [94]:
# 0:정치, 1:경제, 2:사회, 3.생활/문화, 4.세계, 5.IT/과학
classification_dict = {
    0:"정치",
    1:"경제",
    2:"사회",
    3:"생활/문화",
    4:"세계",
    5:"IT/과학",
}
result = model.predict([ X_test[551], X_test[556], X_test[559] ])
for category in result:
    print(classification_dict[category])

IT/과학
사회
정치


##### save model

In [99]:
import pickle

pickle.dump(model, open("classification_model.dat", "wb"))

In [100]:
load_model = pickle.load(open("classification_model.dat", "rb"))

In [114]:
test_str1 = "영국 브렉시트"
test_str2 = "아이폰은 IT 역사에서 길이 남을 애플 제품"
print( classification_dict[ load_model.predict([test_str1])[0] ] )
print( classification_dict[ load_model.predict([test_str2])[0] ] )

세계
IT/과학
